# 1 Изучение данных 

## Загрузка библиотек и чтение данных

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from catboost.utils import get_gpu_device_count
# from sklearn.metrics import (
#     confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, roc_curve
# )
RND_STATE = 42
PATH= 'dataset_train.parquet' 

In [2]:
columns = pd.read_json('features_types.json', typ='series')
data_columns = pd.DataFrame(columns).reset_index()
data_columns.rename(columns = {'index': 'column', 0: 'type'},
                    inplace = True)

In [3]:
data_columns.type.value_counts()

type
numeric            2607
categorical_int     138
categorical_str      31
Name: count, dtype: int64

In [4]:
lst = data_columns.column.tolist()
len(lst)

2776

In [5]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [6]:
# for group in chunker(lst, 100):
#     print(len(group))

In [7]:
# chunks(lst, 100)

In [8]:
def load_parquet_data(path, column_cut):
    list_columns = column_cut.column.tolist()
    list_columns.insert(0, 'id')
    list_columns.insert(1, 'target')
    list_columns.insert(2, 'period')
    df = pd.read_parquet(path, engine='pyarrow', columns=list_columns)
    return list_columns, df
    

In [9]:
 # df_total.info()

In [10]:
# %xdel df_total

df_total.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 702086 entries, 0 to 702085

Columns: 2779 entries, id to markers_945_1_cnt

dtypes: float64(2607), int32(121), int64(31), int8(19), object(1)

memory usage: 14.1+ GB

In [11]:
# %xdel list_columns_1, part_1

In [12]:
# list_columns_1

## Первичный обзор

In [13]:
def initial_overview(data):
    display(data.head(2))
    data.info()
    print(" - " * 30)
    print(" " * 76)
    print("Явных дубликатов в таблице:", data.duplicated().sum())
    print("_" * 32)
    print(" - " * 30)
    print(" " * 76)
    print('Уникальных значений по колонкам:')
    print("_" * 32)
    display(pd.DataFrame(data.nunique()).transpose())
    print(" - " * 30)
    print(" " * 76)
    print("Пропущенных значений по колонкам:")
    print("_" * 32)
    for element in data.columns:
        if data[element].isna().sum() > 0:
            print(element, ' - ', data[element].isna().sum(), 'пропусков, ',
                  round(data[element].isna().sum() * 100 / data.shape[0]), '%')
        

In [14]:
# %%time
# initial_overview(part_6)

## Целевой признак

In [15]:
def pie_plot(data):
    data.target.value_counts().plot.pie(legend=True, labeldistance=1.2, autopct='%1.2f%%', figsize=(3, 3),
                                        title='Соотношение классов по факту покупки'
                                        )
    plt.show()

In [16]:
# pie_plot(part_6)

## Пропуски


* Признаки к удалению (с большим числом пропусков (>50%) + с единственным значением):

In [17]:
def uninform_data(data):
    uninformative_lst = []
    for element in data.columns:
        if (data[element].isna().sum() * 100 / data.shape[0]) > 50 or data[element].nunique() == 1:
            uninformative_lst.append(element)
    return uninformative_lst

In [18]:
%%time
list_columns_1, part_1 = load_parquet_data(PATH, data_columns[:700])  

CPU times: total: 13.4 s
Wall time: 1.37 s


In [19]:
%%time
uninformative_lst_1 = uninform_data(part_1)
len(uninformative_lst_1)

CPU times: total: 5.28 s
Wall time: 5.28 s


25

* Deleting and redefining data sets  (Удаление колонок с большим числом пропусков и  единственным значением, переопределение списка фич):

In [20]:
def deleting_redefining_data(list_columns: list, uninformative_lst: list) -> list:
    redefined_list = list(set(list_columns) - set(uninformative_lst))
    data_columns_rdf = data_columns.loc[data_columns['column'].isin(redefined_list)]
    return data_columns_rdf

In [21]:
# # отбрасываем признаки `over_miss_50` и `std = 0`:
# list_col_not_miss = list(set(list_columns_6) - set(uninformative_lst_6))
# len(list_col_not_miss)

# # Переопределяем дата фрейм с признаками для выгрузки
# data_columns = data_columns.loc[data_columns['column'].isin(list_col_not_miss)]

Переопределение каждого среза фич (всего их 6):

In [22]:
rdf_columns_1 = deleting_redefining_data(list_columns_1, uninformative_lst_1)

In [23]:
rdf_columns_1.type.value_counts()

type
numeric    675
Name: count, dtype: int64

### Числовые признаки

Отбор списка признаков с типом `numeric`, заполнение числовых признаков медианой:

In [24]:
def load_parquet_num(path, rdf_columns):
    numeric_type_data = rdf_columns.loc[rdf_columns['type'] == 'numeric']
    numeric_type_list = numeric_type_data.column.tolist()
    numeric_type_list.insert(0, 'id')
    numeric_type_list.insert(1, 'target')
    numeric_type_list.insert(2, 'period')
    df_num = pd.read_parquet(path, engine='pyarrow', columns=numeric_type_list)
    
    for column in numeric_type_list:
        df_num[column].fillna(df_num[column].median(), inplace=True)
    return numeric_type_list, df_num    

In [25]:
numeric_type_list_1, df_num_1 = load_parquet_num(PATH, rdf_columns_1)

### Категориальные данные

список признаков с типом `categorical_int` и `categorical_str`:

In [26]:
def load_parquet_cat_str(path, rdf_columns):
    cat_type_data = rdf_columns.loc[(rdf_columns['type'] == 'categorical_int') | (rdf_columns['type'] == 'categorical_str')]
    cat_type_list = cat_type_data.column.tolist()
#     cat_type_list.insert(0, 'id')
#     cat_type_list.insert(1, 'target')
#     cat_type_list.insert(2, 'period')
    df_cat = pd.read_parquet(path, engine='pyarrow', columns=cat_type_list)
    return cat_type_list, df_cat

In [27]:
cat_type_list_1, df_cat_1 = load_parquet_cat_str(PATH, rdf_columns_1)

In [28]:
# initial_overview(df_num_1)

**Вывод:**
На данном этапе получено 2 датасета:

* df_num_6 - с числовыми признаками 

* df_cat_6 - с категориальными признаками 

Пропуски в категориальных данных отсутствуют, однако, в признаках с типом `categorical_int`(название заканчивается на _flg) присутствует третье значение: - 1, вероятно, так spark кодирует пропущенные значения.

### Объединение категориальных и численных признаков в один датасет:

In [29]:
# print(df_num_6.shape, df_cat_6.shape)

In [30]:
# df_merge_6 = pd.merge(df_num_6, df_cat_6, left_index=True, right_index=True)

In [31]:
df_merge_1 = df_num_1

In [32]:
# df_merge_2 = pd.merge(df_num_2, df_cat_2, left_index=True, right_index=True)

In [33]:
# df_merge_3 = pd.merge(df_num_3, df_cat_3, left_index=True, right_index=True)

In [34]:
# df_merge_4 = pd.merge(df_num_4, df_cat_4, left_index=True, right_index=True)

In [35]:
# df_merge_5 = pd.merge(df_num_5, df_cat_5, left_index=True, right_index=True)

In [36]:
# df_merge_1.info()

In [37]:
# initial_overview(df_merge_6)

## 4. Обучение модели CatBoostClassifier

Наиболее подходящим способом, в данной ситуации, для выявления важнейших признаков при опредеделении мошенников является ML модель CatBoostClassifier. Она сократит время на настройку параметров, позволит использовать нечисловые факторы вместо предварительной обработки данных, сократит время и усилия на преобразование их в числа. А метоод `catboost select-features` выберет наилучшие фичи из набора данных и избавит от "вредных".

* Отделение таргета от обучающих признаков, сплиттинг на тренировочную и тестовую части:

In [38]:
def targ_feat(data):
    target = data['target']
    features = data.drop(columns=['id', 'target'])
    X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.25, random_state=RND_STATE, stratify=target)
    return features, target, X_train, X_test, y_train, y_test

In [39]:
features, target, X_train, X_test, y_train, y_test = targ_feat(df_merge_1)
print(features.shape, target.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(702086, 676) (702086,) (526564, 676) (175522, 676) (526564,) (175522,)


In [40]:
# features.info()

* Передача признаков конструктору Pool для обработки набора данных:

In [41]:
%%time
eval_set = Pool(data=X_test, label=y_test)

CPU times: total: 15.6 ms
Wall time: 34 ms


* Инициализация модели и подбор лучших признаков с удалением худших:

In [42]:
%%time
CBC = CatBoostClassifier(
    iterations=300,
    loss_function='Logloss',
    eval_metric='AUC:hints=skip_train~false',
    task_type='GPU' if get_gpu_device_count() >= 1 else 'CPU',
    devices='0:1',
    scale_pos_weight=127,
    random_seed=RND_STATE,
    one_hot_max_size=10,
    verbose=False
)


params = {'depth': [2, 3, 4, 5, 6, 7, 8, 9,10],
          'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05]}

CPU times: total: 46.9 ms
Wall time: 65.1 ms


In [43]:
# finding_best_params(X_test_6, y_test_6, X_train_6, y_train_6)

In [44]:
%%time
grid_CBC = GridSearchCV(estimator=CBC, param_grid = params, cv = 2)
grid_CBC.fit(X_train, y_train, eval_set=eval_set)

Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric perio

CPU times: total: 39min 59s
Wall time: 13min 31s


GridSearchCV(cv=2,
             estimator=<catboost.core.CatBoostClassifier object at 0x000001BB214E81F0>,
             param_grid={'depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05]})

In [45]:
%%time

print(" Results from Grid Search " )
print("\n The best estimator:\n",grid_CBC.best_estimator_)
print("\n The best score :\n",grid_CBC.best_score_)
print("\n The best parameters:\n",grid_CBC.best_params_)

 Results from Grid Search 

 The best estimator:

 The best score :
 0.7884074870291171

 The best parameters:
 {'depth': 10, 'learning_rate': 0.05}
CPU times: total: 0 ns
Wall time: 0 ns


## 5. Выявление значимых признаков 

In [46]:
def get_list_imp():
    importances = grid_CBC.best_estimator_.feature_importances_
    data_tuples = list(zip(features.columns.tolist(), importances.tolist()))
    df = pd.DataFrame(data_tuples, columns=['feature','importance'])
    number = int((len(df)*0.1)+1)
    imp_10_per = df.sort_values(by = 'importance', ascending = False).head(number)
    list_imp_feat = imp_10_per.feature.tolist()
    return list_imp_feat

In [47]:
list_imp_feat_1 = get_list_imp()
len(list_imp_feat_1)

68

In [50]:
def preparing_features(PATH, butch):
    list_columns_1, part_1 = load_parquet_data(PATH, butch)  
    uninformative_lst_1 = uninform_data(part_1)
    rdf_columns_1 = deleting_redefining_data(list_columns_1, uninformative_lst_1)
    numeric_type_list_1, df_num_1 = load_parquet_num(PATH, rdf_columns_1)
    cat_type_list_1, df_cat_1 = load_parquet_cat_str(PATH, rdf_columns_1)
    # df_merge_1 = df_num_1
    ## Если есть саt_фичи:
    df_merge_1 = pd.merge(df_num_1, df_cat_1, left_index=True, right_index=True)
    
    features, target, X_train, X_test, y_train, y_test = targ_feat(df_merge_1)
    return features, target, X_train, X_test, y_train, y_test
    # print(features.shape, target.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [51]:
features, target, X_train, X_test, y_train, y_test = preparing_features(PATH, data_columns[700:900])
print(features.shape, target.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(702086, 180) (702086,) (526564, 180) (175522, 180) (526564,) (175522,)


In [52]:
list_imp_feat_2 = get_list_imp()

len(list_imp_feat_2)

19

In [53]:
features, target, X_train, X_test, y_train, y_test = preparing_features(PATH, data_columns[900:1350])
print(features.shape, target.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(702086, 383) (702086,) (526564, 383) (175522, 383) (526564,) (175522,)


In [54]:
list_imp_feat_3 = get_list_imp()
len(list_imp_feat_3)

39

In [55]:
%%time
features, target, X_train, X_test, y_train, y_test = preparing_features(PATH, data_columns[1350:1800])
print(features.shape, target.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(702086, 279) (702086,) (526564, 279) (175522, 279) (526564,) (175522,)
CPU times: total: 22.2 s
Wall time: 10.5 s


In [56]:
%%time
list_imp_feat_4 = get_list_imp()
len(list_imp_feat_4)

CPU times: total: 0 ns
Wall time: 1 ms


28

In [57]:
%%time
features, target, X_train, X_test, y_train, y_test = preparing_features(PATH, data_columns[1800:2300])
print(features.shape, target.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(702086, 168) (702086,) (526564, 168) (175522, 168) (526564,) (175522,)
CPU times: total: 16.1 s
Wall time: 5.41 s


In [58]:
%%time
list_imp_feat_5 = get_list_imp()
len(list_imp_feat_5)

CPU times: total: 0 ns
Wall time: 1 ms


17

In [59]:
%%time
features, target, X_train, X_test, y_train, y_test = preparing_features(PATH, data_columns[2300:])
print(features.shape, target.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(702086, 368) (702086,) (526564, 368) (175522, 368) (526564,) (175522,)
CPU times: total: 24.7 s
Wall time: 11.2 s


In [60]:
%%time
list_imp_feat_6 = get_list_imp()
len(list_imp_feat_6)

CPU times: total: 0 ns
Wall time: 2 ms


37

In [61]:
total_list = list_imp_feat_1 + list_imp_feat_2[1:]+list_imp_feat_3[1:]+ list_imp_feat_4[1:]+ list_imp_feat_5[1:] + list_imp_feat_6[1:]

In [62]:
len(total_list)

203

In [63]:
total_list

['period',
 'markers_706_1_cnt',
 'markers_476_1_cnt',
 'markers_567_1_cnt',
 'markers_508_1_cnt',
 'markers_171_1_cnt',
 'markers_346_1_cnt',
 'markers_387_1_cnt',
 'markers_40_1_cnt',
 'markers_434_1_cnt',
 'markers_721_1_cnt',
 'markers_349_1_cnt',
 'markers_318_1_cnt',
 'markers_438_1_cnt',
 'markers_122_1_cnt',
 'markers_444_1_cnt',
 'markers_324_1_cnt',
 'markers_74_1_cnt',
 'markers_333_1_cnt',
 'markers_622_1_cnt',
 'markers_334_1_cnt',
 'markers_542_1_cnt',
 'markers_103_1_cnt',
 'markers_506_1_cnt',
 'markers_310_1_cnt',
 'markers_242_1_cnt',
 'markers_185_1_cnt',
 'markers_655_1_cnt',
 'markers_7_1_cnt',
 'markers_390_1_cnt',
 'markers_211_1_cnt',
 'markers_328_1_cnt',
 'markers_530_1_cnt',
 'markers_199_1_cnt',
 'markers_3_1_cnt',
 'markers_99_1_cnt',
 'markers_380_1_cnt',
 'markers_576_1_cnt',
 'markers_60_1_cnt',
 'markers_424_1_cnt',
 'markers_65_1_cnt',
 'markers_679_1_cnt',
 'markers_323_1_cnt',
 'markers_701_1_cnt',
 'markers_2_1_cnt',
 'markers_146_1_cnt',
 'markers_